In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import sys
import pandas as pd

from util.scraping

%load_ext autoreload
%autoreload 2

sleep = 8

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
headers = ""

In [3]:
work_id = 19979299
url = 'http://archiveofourown.org/works/' + str(work_id) + "?view_adult=true?view_full_work=true"

r = requests.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

chaps_html = soup.find("div", id="chapters").find_all(class_="userstuff")

[search url](https://archiveofourown.org/works?utf8=%E2%9C%93&work_search%5Bsort_column%5D=kudos_count&include_work_search%5Bfandom_ids%5D%5B%5D=65&work_search%5Bother_tag_names%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Bcrossover%5D=F&work_search%5Bcomplete%5D=T&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bquery%5D=&work_search%5Blanguage_id%5D=en&commit=Sort+and+Filter&tag_id=Avatar%3A+The+Last+Airbender"
)

In [106]:
# map sentiment trend of hurt-comfort fics?

def chap_html_to_str(chap_html):
    all_p = chap_html.find_all("p")
    str_arr = list(map(lambda x: str(x.text).replace("\xa0", " "), all_p))
    str_arr = [st for st in str_arr if st]
    return ' '.join(str_arr)

# could select only common tags here, or just do it afterwards in cleaning
# https://archiveofourown.org/faq/tags?language_id=en#canonicalhow 
# https://fanlore.org/wiki/AO3_Tag_Wrangling
# try to associate tags with canonical tags

the_html = None

def scrape_fic(work_id, by_chapter=True):
    print("getting fic", work_id)
    url = 'http://archiveofourown.org/works/' + str(work_id) + "?view_adult=true?view_full_work=true"

    r = requests.get(url, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 
    
    chaps_html = soup.find("div", id="chapters").find_all(class_="userstuff")
    chaps_clean = list(map(chap_html_to_str, chaps_html))
    
    addl_tags_html = soup.find_all(class_="freeform tags")
    if len(addl_tags_html) >= 2:
        addl_tags_html = addl_tags_html[1].find_all(class_="tag")
        addl_tags = list(map(lambda x: x.text, addl_tags_html))
    else:
        addl_tags = []
    time.sleep(sleep)
    
    #print(chaps_clean)
    #print(addl_tags)
    fic = pd.Series([chaps_clean, addl_tags], index=["text", "addl_tags"])
    
    return fic # if by_chapter else " ".join(chaps_clean)#, notes_clean, summ_html

In [81]:
pd.DataFrame([[1, 2], columns=["text", "tags"])

,text,tags
0,1,2


In [82]:
canonical_tags = {}
tag_map = {}

# https://archiveofourown.org/tags/obligatory%20%22gaang%20finds%20out%20about%20Zuko's%20scar%22%20fic
#def wrangle_tag(tag):

In [111]:
search_url = "https://archiveofourown.org/works?utf8=%E2%9C%93&work_search%5Bsort_column%5D=kudos_count&include_work_search%5Bfandom_ids%5D%5B%5D=65&work_search%5Bother_tag_names%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Bcrossover%5D=F&work_search%5Bcomplete%5D=T&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bquery%5D=&work_search%5Blanguage_id%5D=en&commit=Sort+and+Filter&tag_id=Avatar%3A+The+Last+Airbender"

# single chapter only

test = None

def match_conditions(lang, words, chapters, single_only=False, word_range=(0,0)):
    if lang != "English":
        return False
    if single_only and chapters != 1:
        return False
    if words < word_range[0] or (word_range[1] and words > word_range[1]):
        return False
    return True

def get_id_from_heading(heading_html):
    link = heading_html.find("a")
    if link:
        return link['href'].split("/")[2]
    
def parse_meta(info_html):
    work_id = info_html.get("id").split("_")[1]
    lang = info_html.find_all(class_="language")[1].text
    words = int(info_html.find_all(class_="words")[1].text.replace(",", ""))
    chapters = int(info_html.find_all(class_="chapters")[1].text.split("/")[0])
    
    return {"work_id": work_id, "lang": lang, "words": words, "chapters": chapters}
    
# test running out of works
def get_work_ids(url, count, page=1, single_only=False, word_range=(0,0)):
    print(f"getting page {page}, {count} works left")
    url_page = url if page == 1 else url + "&page=" + str(page)

    r = requests.get(url_page, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    info_list = soup.find_all(class_="blurb")
    
    if not info_list:
        return []
    work_info = [parse_meta(info_html) for info_html in info_list]
    work_ids = [x["work_id"] for x in work_info if match_conditions(x["lang"], x["words"], x["chapters"], single_only=single_only, word_range=word_range)]

    if count <= len(work_ids):
        return work_ids[:count]    
    time.sleep(sleep)
    return work_ids + get_work_ids(url, count-len(work_ids), page+1, single_only=single_only, word_range=word_range)

def get_work_info(url, count, page=1, single_only=False, word_range=(0,0)):
    print(f"getting page {page}, {count} works left")
    url_page = url if page == 1 else url + "&page=" + str(page)

    r = requests.get(url_page, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    info_list = soup.find_all(class_="blurb")
    
    if not info_list:
        return []
    work_info = [parse_meta(info_html) for info_html in info_list]
    work_info = [x for x in work_info if match_conditions(x["lang"], x["words"], x["chapters"], single_only=single_only, word_range=word_range)]

    if count <= len(work_info):
        print("finished")
        return work_info[:count]    
    time.sleep(sleep)
    return work_info + get_work_info(url, count-len(work_info), page+1, single_only=single_only, word_range=word_range)

# options: single chapter only, word count

In [110]:
fics_info = get_work_info(search_url, 100, single_only=True)

getting page 1, 100 works left
getting page 2, 83 works left
getting page 3, 67 works left
getting page 4, 54 works left
getting page 5, 40 works left
getting page 6, 25 works left
getting page 7, 8 works left


In [112]:
info_df = pd.DataFrame.from_dict(fics_info)

In [113]:
text_df = pd.DataFrame(info_df["work_id"].apply(scrape_fic))
final_df = pd.concat([info_df, text_df], axis=1)

getting fic 14921627
getting fic 15753762
getting fic 14510448
getting fic 21526240
getting fic 19979299
getting fic 1078856
getting fic 1938096
getting fic 10990518
getting fic 21990778
getting fic 21086960
getting fic 19416352
getting fic 1874571
getting fic 15923450
getting fic 330627
getting fic 16964727
getting fic 24400723
getting fic 22011667
getting fic 23953027
getting fic 23439082
getting fic 16621667
getting fic 23346418
getting fic 15925412
getting fic 21383983
getting fic 956643
getting fic 21327295
getting fic 23320117
getting fic 24674929
getting fic 21455932
getting fic 1214218
getting fic 20103208
getting fic 10771164
getting fic 24100726
getting fic 19255837
getting fic 6441391
getting fic 20655671
getting fic 19856629
getting fic 24596833
getting fic 23132179
getting fic 22907446
getting fic 33751
getting fic 24560689
getting fic 15861333
getting fic 24056428
getting fic 22105117
getting fic 17385011
getting fic 24246931
getting fic 20014126
getting fic 2250771
getti

In [114]:
final_df.to_pickle("avatar_fic_100_data.pickle")

In [ ]:
info_dict["work_id"].apply(scrape_fic)

In [ ]:
work_dict = pd.DataFrame(info_dict["work_id"].ap

In [ ]:
texts = []
for info in single_chapter_fics: #avatar_single_chap:
    texts.append(scrape_fic(info["work_id"]))
    #try:
    #    texts.append(scrape_fic(id))
    #except Exception as e:
    #    print(f"error with work {id}", e)

In [93]:

df = pd.DataFrame(texts)
df.to_csv("texts_2.csv")


In [95]:
#df.to_pickle("50_texts.pickle")

In [ ]:
scrape_fic(21526240)

In [26]:
soup = BeautifulSoup(html, 'html.parser')
chaps_html = soup.find("div", id="chapters").find_all(class_="userstuff")

KeyError: 22

In [1]:
chap_html = chaps_html[0]
all_p = chap_html.find_all("p")
str_arr = list(map(lambda x: str(x.text).replace("\xa0", "").replace("\n", "").replace("\", ""), all_p))
str_arr = [st for st in str_arr if st]



SyntaxError: EOL while scanning string literal (<ipython-input-1-8900984dca93>, line 3)